<a href="https://colab.research.google.com/github/chminPark/ml-python/blob/master/PretrainedVGG%EA%B8%B0%EB%B0%98_%EA%B8%B0%EB%B0%98%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Pytorch로 개발하기 위한 Library
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

import numpy as np

torch.__version__

'2.1.0+cu118'

In [ ]:
!gdown --id 1L3aXv9A2bn8G9orLqjVWOE_1OY4ycdjs
!mkdir 'chip_data'
!unzip -q chip_data.zip -d chip_data
!rm chip_data.zip

In [ ]:
import csv
import os
import pandas as pd
"""
Args:
  directory_string: 이미지가 저장되어 있는 폴더 Path
  output_csv_name: csv 파일 이름
Returns:
  csv file
"""
def build_csv(directory_string, output_csv_name):

    directory = directory_string
    class_list = os.listdir(directory)
    class_list.sort()

    dict_map = {
        'crack, scratch, stamp' : 1,
        'particle' : 2,
        'titled' : 3,
        "와이어끊어짐" : 4,
        "정상": 0
    }
    with open(output_csv_name, 'w', newline='') as csvfile:

        ### 실습 : CSV 파일 Object 생성
        writer = csv.writer(csvfile, delimiter=',')
        ############################################
        writer.writerow(['file_name', 'file_path', 'class_name', 'class_index']) # CSV의 column 이름을 지정

        ###### 각 folder에 들어가서 각 이미지의 이름을 가져옴
        for class_name in class_list:
          class_path = os.path.join(directory, class_name)
          file_list = os.listdir(class_path) # 해당 파일 내부의 이미지를 확보
          for file_name in file_list:
              file_path = os.path.join(directory, class_name, file_name) #concatenate class folder dir, class name and file name
              writer.writerow([file_name, file_path, class_name, dict_map[class_name]]) #write the file path and class name to the csv file
        #############################

    return

train_folder = os.path.join(os.getcwd(), 'chip_data')
build_csv(train_folder, 'train.csv')
all_df = pd.read_csv('train.csv')


In [ ]:
all_df

,file_name,file_path,class_name,class_index
0,Sat Nov 25 15-20-27.jpg,"/content/chip_data/crack, scratch, stamp/Sat N...","crack, scratch, stamp",1
1,Sat Nov 25 15-20-32.jpg,"/content/chip_data/crack, scratch, stamp/Sat N...","crack, scratch, stamp",1
2,Sat Nov 25 13-22-27.jpg,"/content/chip_data/crack, scratch, stamp/Sat N...","crack, scratch, stamp",1
3,Sat Nov 25 15-24-11.jpg,"/content/chip_data/crack, scratch, stamp/Sat N...","crack, scratch, stamp",1
4,Sat Nov 25 15-24-15.jpg,"/content/chip_data/crack, scratch, stamp/Sat N...","crack, scratch, stamp",1
...,...,...,...,...
850,Sat Nov 25 14-05-57.jpg,/content/chip_data/정상/Sat Nov 25 14-05-57.jpg,정상,0
851,Sat Nov 25 14-08-38.jpg,/content/chip_data/정상/Sat Nov 25 14-08-38.jpg,정상,0
852,Sat Nov 25 13-44-32.jpg,/content/chip_data/정상/Sat Nov 25 13-44-32.jpg,정상,0
853,Sat Nov 25 13-54-56.jpg,/content/chip_data/정상/Sat Nov 25 13-54-56.jpg,정상,0


In [ ]:
all_df['class_index'].unique()

array([1, 2, 3, 4, 0])

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(all_df, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
train['class_index'].value_counts(), test['class_index'].value_counts()

(0    369
 1     99
 2     80
 4     79
 3     57
 Name: class_index, dtype: int64,
 0    88
 1    28
 4    22
 3    18
 2    15
 Name: class_index, dtype: int64)

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(all_df, test_size=0.2, random_state=42, shuffle=True, stratify=all_df['class_index'])
train['class_index'].value_counts(), test['class_index'].value_counts()

(0    365
 1    102
 4     81
 2     76
 3     60
 Name: class_index, dtype: int64,
 0    92
 1    25
 4    20
 2    19
 3    15
 Name: class_index, dtype: int64)

In [ ]:
# save df to csv
train.to_csv("train.csv", index=0)
test.to_csv("test.csv", index=0)

In [ ]:
pd.read_csv("train.csv")

,file_name,file_path,class_name,class_index
0,Sat Nov 25 15-17-34.jpg,/content/chip_data/와이어끊어짐/Sat Nov 25 15-17-34.jpg,와이어끊어짐,4
1,Sat Nov 25 15-38-05.jpg,/content/chip_data/particle/Sat Nov 25 15-38-0...,particle,2
2,Sat Nov 25 14-17-43.jpg,/content/chip_data/정상/Sat Nov 25 14-17-43.jpg,정상,0
3,Sat Nov 25 15-24-12.jpg,"/content/chip_data/crack, scratch, stamp/Sat N...","crack, scratch, stamp",1
4,Sat Nov 25 14-05-24.jpg,/content/chip_data/정상/Sat Nov 25 14-05-24.jpg,정상,0
...,...,...,...,...
679,Sat Nov 25 13-28-25.jpg,/content/chip_data/정상/Sat Nov 25 13-28-25.jpg,정상,0
680,Sat Nov 25 15-15-11.jpg,/content/chip_data/와이어끊어짐/Sat Nov 25 15-15-11.jpg,와이어끊어짐,4
681,Sat Nov 25 15-41-02.jpg,/content/chip_data/particle/Sat Nov 25 15-41-0...,particle,2
682,Sat Nov 25 15-25-58.jpg,"/content/chip_data/crack, scratch, stamp/Sat N...","crack, scratch, stamp",1


# Custom데이터를 활용하기 위한 Dataset을 선언
- pytorch의 `dataloader`를 이용하기 위해서는 `torch.utils.data.Dataset` 클래스를 상속한 클래스의 선언이 필요하다.
- 클래스 내에 `__init__`, `__getitem__`, `__len__`의 3개의 메소드를 선언하여 오버라이드한다.

In [ ]:
import cv2
import torch
from torch.utils.data import Dataset

class semiconductorDataset(Dataset): # inheritin from Dataset class

    def __init__(self, csv_file, root_dir="", transform=None):
        self.annotation_df = pd.read_csv(csv_file)
        self.root_dir = root_dir # root directory of images, leave "" if using the image path column in the __getitem__ method
        self.transform = transform

    def __len__(self):
        return len(self.annotation_df) # return length (numer of rows) of the dataframe

    ############ 실습 : annotation_df에 있는 이미지를 읽어 들어서, 변경한후
    ############        읽어 들인 값을 return 하는 함수를 작성한다.
    def __getitem__(self, idx):

        ##### Image 읽기
        image_path = os.path.join(self.root_dir, self.annotation_df.iloc[idx, 1]) #use image path column (index = 1) in csv file
        image = cv2.imread(image_path) # read image by cv2
        #### 이미지를 Channel순서를 변경
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert from BGR to RGB for matplotlib
        #### 이미지 Transform
        if self.transform:
            image = self.transform(image)

        #### 이미지에 대한 추가 정보 확인
        class_name = self.annotation_df.iloc[idx, 2] # use class name column (index = 2) in csv file
        class_index = self.annotation_df.iloc[idx, 3] # use class index column (index = 3) in csv file

        return image, class_name, class_index

In [ ]:
# https://pytorch.org/vision/stable/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py
from torchvision import transforms
import matplotlib.pyplot as plt
import random

transform = transforms.Compose([
    transforms.ToTensor(), # PIL Image를 Tensor로 변경
    transforms.Resize((100,100), antialias=True), # 크기 변경
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), # Normalize
    transforms.RandomAdjustSharpness(sharpness_factor=2), # randomly adjusts the sharpness
    transforms.RandomAutocontrast() # 임의로 Contrast변경
])

transform_test = transforms.Compose([
    transforms.ToTensor(), # PIL Image를 Tensor로 변경
    transforms.Resize((100,100), antialias=True), # 크기 변경
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), # Normalize
])

##### 실습 : train_dataset 선언
##### 실습 : test_dataset 선언
train_dataset = semiconductorDataset(csv_file='train.csv', root_dir="", transform=transform)
test_dataset = semiconductorDataset(csv_file='test.csv', root_dir="", transform=transform_test)

# Batch 단위 학습을 위한 DataLoader 선언
 - Dataset내부의 샘플들을 batch 크기로 추출
 - Batch Size는 1step에 들어간 데이터의 개수
 - Epoch 마다 데이터를 섞어(Shuffle) Overfitting을 방지
 - 병렬처리를 지원하여 데이터 검색 속도를 향상


In [ ]:
#### 실습 : torch.utils.DataLoader를 이용하여 데이터를 load. Batch 크기를 10으로 한다.
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=10, shuffle=True, num_workers=2)

for i, data in enumerate(train_dataloader):
  images, class_name, labels = data
  print(images.shape, labels.shape)

  # 5번만 데이터를 load하고 멈춘다
  if i > 3:
    break

torch.Size([10, 3, 100, 100]) torch.Size([10])
torch.Size([10, 3, 100, 100]) torch.Size([10])
torch.Size([10, 3, 100, 100]) torch.Size([10])
torch.Size([10, 3, 100, 100]) torch.Size([10])
torch.Size([10, 3, 100, 100]) torch.Size([10])


# Pretrained 모델을 활용한다

In [ ]:
from torchvision import models
import torch

vgg16_pretrained = models.vgg16(weights='VGG16_Weights.IMAGENET1K_V1')

# 최종 결과가 1000개로 나온다. 우리의 데이터는 11개의 class로 변경해야 한다.
print(vgg16_pretrained.classifier)
print(vgg16_pretrained.classifier[-1])
vgg16_pretrained.classifier[3] = nn.Linear(in_features=4096, out_features=1000, bias=True)
vgg16_pretrained.classifier[6] = nn.Linear(in_features=1000, out_features=11, bias=True)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)
Linear(in_features=4096, out_features=1000, bias=True)


In [ ]:
# 일부 layer frozon  하기
for name, params in vgg16_pretrained.named_parameters():
  # print(name, params.requires_grad)

  if 'features' in name:
    idx = int(name.split(".")[1])
    if idx <= 5:
      params.requires_grad = False

In [ ]:
def train(model, data_loader, criterion, optimizer, n_epoch):

  model.train() #
  for epoch in range(n_epoch):
    running_loss = 0
    ### 실습 : Custom dataloader 에서 값을 가져온다
    for i, (images, _, labels) in enumerate(data_loader):


      images, labels = images.cuda(), labels.cuda()
      outputs = model(images)
      loss = criterion(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
      if (i + 1) % 100 == 0:
        print('iteration: [{}/{}]'.format(i + 1, len(data_loader)))

    print('Epoch {}, loss = {:.3f}'.format(epoch + 1, running_loss/len(data_loader)))

In [ ]:
def eval(model, data_loader):
  #### 실습 : 평가를 위해서는 eval()을 선언
  model.eval()
  total = 0
  correct = 0

  preds = []
  trues = []
  #### 실습 : Gradient를 타지 않아야 한다
  with torch.no_grad():
    for images, _, labels in data_loader:
      images, labels = images.cuda(), labels.cuda()
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      preds.append(predicted.detach().cpu().numpy())
      trues.append(labels.detach().cpu().numpy())

    accuracy = 100 * correct / total

  print('Test Accuracy: {}%'.format(accuracy))
  return preds, trues


In [ ]:
import numpy as np
def reset_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
reset_seed(2020)
criterion = nn.CrossEntropyLoss()
vgg16 = vgg16_pretrained.to("cuda")
optimizer = optim.Adam(params=vgg16.parameters(), lr=0.00003)

train(vgg16, train_dataloader, criterion, optimizer, n_epoch=10)

Epoch 1, loss = 1.308
Epoch 2, loss = 0.606
Epoch 3, loss = 0.443
Epoch 4, loss = 0.324
Epoch 5, loss = 0.264
Epoch 6, loss = 0.263
Epoch 7, loss = 0.202
Epoch 8, loss = 0.210
Epoch 9, loss = 0.177
Epoch 10, loss = 0.183


# 정확도를 평가해 봅시다

In [ ]:
#### 실습 : torch.utils.DataLoader를 이용하여 데이터를 load. Batch 크기를 10으로 한다.
test_dataloader = torch.utils.data.DataLoader(test_dataset,batch_size=10, shuffle=False, num_workers=2, drop_last=True)

for i, data in enumerate(test_dataloader):
  images, class_name, labels = data
  print(images.shape, labels.shape)

  # 5번만 데이터를 load하고 멈춘다
  if i > 1:
    break

torch.Size([10, 3, 100, 100]) torch.Size([10])
torch.Size([10, 3, 100, 100]) torch.Size([10])
torch.Size([10, 3, 100, 100]) torch.Size([10])


In [ ]:
preds_r, trues_r = eval(vgg16, train_dataloader)

Test Accuracy: 94.15204678362574%


In [ ]:
preds, trues = eval(vgg16, test_dataloader)

Test Accuracy: 87.6470588235294%


In [ ]:
prediction_label = np.stack(preds).flatten()
test_labels = np.stack(trues).flatten()
prediction_label, test_labels

(array([3, 3, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 2, 0, 2, 0, 3,
        2, 4, 0, 0, 0, 3, 0, 0, 1, 4, 0, 0, 0, 2, 2, 4, 1, 1, 0, 0, 0, 0,
        3, 0, 0, 0, 1, 0, 3, 0, 4, 2, 3, 1, 2, 0, 1, 4, 0, 0, 0, 3, 3, 4,
        2, 4, 0, 0, 0, 0, 0, 0, 1, 3, 1, 3, 0, 0, 2, 1, 0, 4, 2, 0, 0, 1,
        0, 0, 1, 0, 0, 2, 1, 4, 0, 3, 4, 0, 0, 0, 0, 0, 4, 2, 4, 0, 0, 1,
        0, 1, 0, 4, 0, 0, 0, 0, 3, 1, 0, 0, 4, 4, 3, 0, 4, 0, 1, 0, 4, 0,
        3, 0, 0, 3, 0, 0, 3, 2, 0, 0, 2, 3, 0, 0, 0, 0, 0, 3, 0, 4, 0, 1,
        0, 1, 1, 3, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 4]),
 array([0, 0, 0, 1, 0, 0, 1, 3, 0, 1, 0, 0, 1, 0, 0, 2, 0, 2, 0, 2, 2, 3,
        2, 4, 0, 0, 0, 3, 0, 0, 1, 4, 1, 0, 0, 2, 2, 4, 1, 1, 0, 0, 1, 0,
        3, 0, 0, 0, 1, 0, 3, 0, 4, 2, 0, 0, 2, 0, 0, 4, 0, 0, 0, 3, 3, 4,
        2, 4, 0, 0, 0, 0, 3, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 4, 2, 0, 0, 1,
        0, 0, 1, 0, 0, 2, 1, 4, 0, 3, 4, 0, 0, 1, 0, 0, 4, 2, 4, 0, 0, 1,
        0, 1, 0, 4, 0, 0, 0, 0, 0, 1, 0, 0, 4, 4, 3, 0

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_labels, prediction_label))

              precision    recall  f1-score   support

           0       0.89      0.88      0.89        92
           1       0.87      0.80      0.83        25
           2       1.00      0.89      0.94        18
           3       0.62      0.87      0.72        15
           4       1.00      0.95      0.97        20

    accuracy                           0.88       170
   macro avg       0.88      0.88      0.87       170
weighted avg       0.89      0.88      0.88       170

